In [1]:
#### LIBRARIES AND FUNCTIONS
using DataFrames
using CSV
using JLD2
using Plots
using StatsBase
using LaTeXStrings

In [2]:
n_sim = 1:32; 
batches = ["FirstBatch" "SecondBatch" "ThirdBatch"]
T = 500

500

In [33]:
path = "./data/FirstBatch/data1/Data.jld2"
X = JLD2.load(path)
parameters = X["parameters"]
N,p,tmin,tmax,Δt,t,L,θ,σ,it_network,it_brownian = parameters

θs = zeros(length(n_sim)) 
OrdPar = zeros(length(n_sim)) 
OrdPar_Min = zeros(length(n_sim))
OrdPar_Max = zeros(length(n_sim))
N_network = zeros(length(n_sim))

for index in 1:32
    OrdPar_batch = zeros(length(t[t.>=T]))
    OrdPar_Min_batch = 0
    OrdPar_Max_batch = 0
    l_tot = 0
    for (index_str,str) in enumerate(batches)
        path = "./data/"*str*"/data"*string(index)*"/Data.jld2"
        try 
            X = JLD2.load(path)
            ord_par = X["order_parameter"]
            parameters = X["parameters"]
            N,p,tmin,tmax,Δt,t,L,θ,σ,it_network,it_brownian = parameters
            
            ord_par_asymp = ord_par[ t .>= T ] 
            OrdPar_batch .+= ord_par_asymp
            OrdPar_Min_batch += minimum(ord_par_asymp)  
            OrdPar_Max_batch += maximum(ord_par_asymp) 
            l_tot += 1
            θs[index] = θ
            
        catch
            println(path*" does not exist")
        end
    end
    #OrdPar_batch ./= l_tot
    OrdPar[index] = mean(OrdPar_batch ./ l_tot)
    OrdPar_Min[index] = OrdPar_Min_batch ./ l_tot
    OrdPar_Max[index] = OrdPar_Max_batch ./ l_tot
    N_network[index] = l_tot
end

./data/FirstBatch/data3/Data.jld2 does not exist
./data/FirstBatch/data4/Data.jld2 does not exist
./data/FirstBatch/data5/Data.jld2 does not exist


In [35]:
θs_ren = θs./(σ^2/p)
plot(θs_ren,OrdPar,color=:blue,linewidth=1.5,label="",dpi=400)
plot!(θs_ren,OrdPar_Min,color=:red,linestyle=:dash,linewidth=0.7,label="")
plot!(θs_ren,OrdPar_Max,color=:red,linestyle=:dash,linewidth=0.7,label="")
xlabel!(L"\theta/\theta_c",xguidefontsize=18)
ylabel!(L"|r|",yguidefontsize=18)
#savefig("Kuramoto_ER.png")

"/Users/niccolo/Desktop/Graphs/Kuramoto_ER.png"